# SISTER workflow

In [24]:
import json
import os
import xml.etree.ElementTree as ET
import pandas as pd
import IPython
from itertools import groupby

# Import warnings module and ignore warnings in output below
import warnings
warnings.filterwarnings("ignore")

# Import and initialize MAAP class
from maap.maap import MAAP
maap = MAAP(maap_host="sister-api.imgspec.org")

### Create unique scene identifier

In [25]:
granules = ['https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/raw/PRS_L1_STD_OFFL_20200911170127_20200911170131_0001.zip',
            'https://popo.jpl.nasa.gov/avcl/y17_data/f170507t01p00r11.tar.gz',
            'https://avng.jpl.nasa.gov/avng/y18_data/ang20180429t045252.tar.gz',
            'https://sister-ops-workspace.s3.us-west-2.amazonaws.com/desis/raw/DESIS-HSI-L1C-DT0327290940_002-20190609T195930-V0210.zip']

meta = 'TAG_TESTING'

scenes = []

crid = "985"

for l1_granule in granules:
    
    landsat = 'None'   

    base_name = os.path.basename(l1_granule)

    if base_name.startswith('DESIS'):
        sensor = 'DESIS'
        datetime = base_name[31:46]

    elif base_name.startswith('PRS'):
        sensor = 'PRISMA'
        datetime = base_name[16:24] + 'T' + base_name[24:30]
        landsat='https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/landsat_reference/PRS_%s_landsat.tar.gz' % base_name[16:50]

    elif base_name.startswith('ang'):
        sensor = 'AVNG'
        datetime = base_name[3:18]

    elif base_name.startswith('f'):
        sensor = 'AVCL'
        ''' AVIRIS classic filenames do not contain acquisition times,to be consistent with other
            sensors and to ensure identifier codes are unique a time string is created using other
            numbers in the filename            
        '''     

        datetime = "20%sT%s%s%s" % (base_name[1:7],
                                    base_name[8:10],
                                    base_name[11:13],
                                    base_name[14:16])
    else:
        raise ValueError('Unrecognized L1 datafile')

    job_args = {'sensor': sensor,
                'datetime': datetime,
                 'crid' : crid}
    job_args['preprocess'] = {'raw_dataset': l1_granule,
                              'landsat_dataset' : landsat}

    print(job_args)
    
    scenes.append(job_args)


{'sensor': 'PRISMA', 'datetime': '20200911T170127', 'crid': '985', 'preprocess': {'raw_dataset': 'https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/raw/PRS_L1_STD_OFFL_20200911170127_20200911170131_0001.zip', 'landsat_dataset': 'https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/landsat_reference/PRS_20200911170127_20200911170131_0001_landsat.tar.gz'}}
{'sensor': 'AVCL', 'datetime': '20170507T010011', 'crid': '985', 'preprocess': {'raw_dataset': 'https://popo.jpl.nasa.gov/avcl/y17_data/f170507t01p00r11.tar.gz', 'landsat_dataset': 'None'}}
{'sensor': 'AVNG', 'datetime': '20180429t045252', 'crid': '985', 'preprocess': {'raw_dataset': 'https://avng.jpl.nasa.gov/avng/y18_data/ang20180429t045252.tar.gz', 'landsat_dataset': 'None'}}
{'sensor': 'DESIS', 'datetime': '20190609T195930', 'crid': '985', 'preprocess': {'raw_dataset': 'https://sister-ops-workspace.s3.us-west-2.amazonaws.com/desis/raw/DESIS-HSI-L1C-DT0327290940_002-20190609T195930-V0210.zip', 'landsat_dataset

## Step 1. Preprocess

In [26]:
for i,scene in enumerate(scenes):
        
    if scene['sensor'] == 'AVCL':
        queue="sister-job_worker-32gb"
    else:
        queue="sister-job_worker-16gb"
    
    identifier = f'SISTER_{scene["sensor"]}_L1B_RDN_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    preprocess_job_response = maap.submitJob(
        algo_id = "sister-preprocess",
        version = "2.0.0",
        raw_dataset = scene['preprocess']['raw_dataset'],
        landsat_dataset = scene['preprocess']['landsat_dataset'],
        crid = scene['crid'],
        publish_to_cmr = False,
        cmr_metadata={},
        queue=queue,
        identifier= identifier)

    print(f'Identifier: {identifier}')
    print(f'Submission status: {preprocess_job_response.status}')
    print(f'Job ID: {preprocess_job_response.id}')
          
    scenes[i]['preprocess']['job_id'] = preprocess_job_response.id
    

Identifier: SISTER_PRISMA_L1B_RDN_20200911T170127_985_TAG_TESTING
Submission status: success
Job ID: b09a2589-62a2-4d33-b7a6-5b6b5ccf8ec4
Identifier: SISTER_AVCL_L1B_RDN_20170507T010011_985_TAG_TESTING
Submission status: success
Job ID: 550ca2fd-17c4-4279-9d07-07a46268f807
Identifier: SISTER_AVNG_L1B_RDN_20180429t045252_985_TAG_TESTING
Submission status: success
Job ID: 6cd47ebe-a557-4943-aa4d-0ba185cf817c
Identifier: SISTER_DESIS_L1B_RDN_20190609T195930_985_TAG_TESTING
Submission status: success
Job ID: e3d597ee-ecb9-4264-96bf-bb34f956497d


## Step 2. ISOFIT


In [27]:
for i,scene in enumerate(scenes):
        
    if scene['sensor'] == 'AVCL':
        segmentation_size = 100
    else:
        segmentation_size = 100
    
    identifier = f'SISTER_{scene["sensor"]}_L2A_RFL_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    preprocess_id = scene['preprocess']['job_id'] 
    
    preprocess_result= [ x for x in maap.getJobResult(preprocess_id).outputs if x.startswith("s3://s3.") and "RDN" in x]
    preprocess_result.sort()
    l1b_rdn,l1b_loc, l1b_obs = preprocess_result
    
    scene['preprocess']['radiance_dataset'] =l1b_rdn
    scene['preprocess']['location_dataset'] =l1b_loc
    scene['preprocess']['observation_dataset'] =l1b_obs

    isofit_job_response = maap.submitJob(
                                    algo_id="sister-isofit",
                                    version="2.0.0",
                                    radiance_dataset=l1b_rdn,
                                    location_dataset = l1b_loc,
                                    observation_dataset = l1b_obs,
                                    segmentation_size = segmentation_size,
                                    n_cores=32,
                                    crid = scene['crid'],
                                    publish_to_cmr=False,
                                    cmr_metadata={},
                                    queue="sister-job_worker-32gb",
                                    identifier= identifier)

    print(f'Identifier: {identifier}')
    print(f'Submission status: {isofit_job_response.status}')
    print(f'Job ID: {isofit_job_response.id}')
    
    scene['isofit']  = {'job_id' : isofit_job_response.id}


Identifier: SISTER_PRISMA_L2A_RFL_20200911T170127_985_TAG_TESTING
Submission status: success
Job ID: a8390b87-949a-4430-a09a-7522f1f1ccc0
Identifier: SISTER_AVCL_L2A_RFL_20170507T010011_985_TAG_TESTING
Submission status: success
Job ID: 250b9971-ae8d-45b5-b05d-99f4d8cae554
Identifier: SISTER_AVNG_L2A_RFL_20180429t045252_985_TAG_TESTING
Submission status: success
Job ID: bc812462-2b5f-44ae-a808-2f82abbc2e59
Identifier: SISTER_DESIS_L2A_RFL_20190609T195930_985_TAG_TESTING
Submission status: success
Job ID: b952c3c3-cf25-4528-9f41-8656cb444a68


## Step 3. Spectral resample

In [28]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2A_RSRFL_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    isofit_id = scene['isofit']['job_id'] 

    iso_result= [ x for x in maap.getJobResult(isofit_id).outputs if x.startswith("s3://s3.") and "RFL" in x]
    l2a_rfl,l2a_unc = iso_result
        
    scene['isofit']['reflectance_dataset'] =l2a_rfl
    scene['isofit']['uncertainty_dataset'] =l2a_unc

    resample_job_response = maap.submitJob(
                                            algo_id="sister-resample",
                                            version="2.0.0",
                                            reflectance_dataset= l2a_rfl,
                                            uncertainty_dataset= l2a_unc,
                                            crid = scene['crid'],
                                            publish_to_cmr=False,
                                            cmr_metadata={},
                                            queue="sister-job_worker-16gb",
                                            identifier=identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % resample_job_response.status)
    print('Job ID: %s' % resample_job_response.id)
    scene['resample']  = {'job_id' : resample_job_response.id}


Identifier: SISTER_PRISMA_L2A_RSRFL_20200911T170127_985_TAG_TESTING
Submission status: success
Job ID: f8f6092f-6048-40b6-b955-15c18a73182c
Identifier: SISTER_AVCL_L2A_RSRFL_20170507T010011_985_TAG_TESTING
Submission status: success
Job ID: 25c5e0ce-e349-4926-9c76-bdd07f083bc3
Identifier: SISTER_AVNG_L2A_RSRFL_20180429t045252_985_TAG_TESTING
Submission status: success
Job ID: 5ac85cf9-3546-47d9-b470-aeb8438e1ffb
Identifier: SISTER_DESIS_L2A_RSRFL_20190609T195930_985_TAG_TESTING
Submission status: success
Job ID: c4c201cb-bb78-43de-abeb-08ee3384ea04


## Step 3. Reflectance correction

In [29]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2A_CORFL_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    resample_id = scene['resample']['job_id'] 
    resample_result= [ x for x in maap.getJobResult(resample_id).outputs if x.startswith("s3://s3.") and "RSRFL" in x]
    l2a_rsrfl,l2a_rsunc = resample_result
    
    scene['resample']['reflectance_dataset'] =l2a_rsrfl
    scene['resample']['uncertainty_dataset'] =l2a_rsunc



    rfl_corr_job_response = maap.submitJob(
                                            algo_id="sister-reflect_correct",
                                            version="2.0.0",
                                            observation_dataset= scene['preprocess']['observation_dataset'],
                                            reflectance_dataset= l2a_rsrfl,
                                            crid = scene['crid'],
                                            publish_to_cmr=False,
                                            cmr_metadata={},
                                            queue="sister-job_worker-16gb",
                                            identifier=identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % rfl_corr_job_response.status)
    print('Job ID: %s' % rfl_corr_job_response.id)
    scene['reflect_correct']  = {'job_id' : rfl_corr_job_response.id}


Identifier: SISTER_PRISMA_L2A_CORFL_20200911T170127_985_TAG_TESTING
Submission status: success
Job ID: 00fdc90f-0b8f-45d2-a3a7-5c4dbacc3bdf
Identifier: SISTER_AVCL_L2A_CORFL_20170507T010011_985_TAG_TESTING
Submission status: success
Job ID: fafb5143-5e82-4564-8f4c-6d44ccaffd61
Identifier: SISTER_AVNG_L2A_CORFL_20180429t045252_985_TAG_TESTING
Submission status: success
Job ID: 8dac0cab-08ef-4cc8-8180-67dab670e5e6
Identifier: SISTER_DESIS_L2A_CORFL_20190609T195930_985_TAG_TESTING
Submission status: success
Job ID: d2c8fae5-2fff-4ac9-a286-3e7754b001cf


## Step 4. Fractional Cover

In [30]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2B_FRCOV_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    correct_id = scene['reflect_correct']['job_id'] 
    correct_result= [ x for x in maap.getJobResult(correct_id).outputs if x.startswith("s3://s3.") and "CORFL" in x]
    l2a_corfl =  correct_result[0]
    scene['reflect_correct']['reflectance_dataset'] = l2a_corfl

    frcover_job_response = maap.submitJob(
                                        algo_id="sister-fractional-cover",
                                        version="1.0.0",
                                        reflectance_dataset=l2a_corfl,
                                        n_cores= 20,
                                        refl_scale= 1,
                                        normalization = 'brightness',
                                        crid = scene['crid'],
                                        publish_to_cmr=False,
                                        cmr_metadata={},
                                        queue="sister-job_worker-32gb",
                                        identifier= identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % frcover_job_response.status)
    print('Job ID: %s' % frcover_job_response.id)
    scene['frcover']  = {'job_id' : frcover_job_response.id}

Identifier: SISTER_PRISMA_L2B_FRCOV_20200911T170127_985_TAG_TESTING
Submission status: success
Job ID: b8980909-0aec-490e-b6c2-76ed0c74181f
Identifier: SISTER_AVCL_L2B_FRCOV_20170507T010011_985_TAG_TESTING
Submission status: success
Job ID: d7e63792-0e70-42de-ab0f-027eba77510a
Identifier: SISTER_AVNG_L2B_FRCOV_20180429t045252_985_TAG_TESTING
Submission status: success
Job ID: 0c35ded0-5d03-40d9-a675-7ca760dde2aa
Identifier: SISTER_DESIS_L2B_FRCOV_20190609T195930_985_TAG_TESTING
Submission status: success
Job ID: 013fa4e1-2728-48cf-b31a-751b5cdf5886


## Step 6a. Vegetation biochemistry

In [32]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2B_VEGBIOCHEM_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    frcover_id = scene['frcover']['job_id'] 
    frcover_result= [x for x in maap.getJobResult(frcover_id).outputs if x.startswith("s3://s3.") and "FRCOV" in x]
    l2b_frcov =  frcover_result[0]                   
    scene['frcover']['frcover_dataset'] = l2b_frcov

    vegbiochem_job_response = maap.submitJob(
                            algo_id="sister-trait_estimate",
                            version="1.0.0",
                            reflectance_dataset= scene['reflect_correct']['reflectance_dataset'],
                            frcov_dataset=l2b_frcov,
                            veg_cover = 0.5,
                            crid = scene['crid'],
                            publish_to_cmr=False,
                            cmr_metadata={},
                            queue="sister-job_worker-16gb",
                            identifier=identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % vegbiochem_job_response.status)
    print('Job ID: %s' % vegbiochem_job_response.id)
    scene['vegbiochem']  = {'job_id' : vegbiochem_job_response.id}

Identifier: SISTER_PRISMA_L2B_VEGBIOCHEM_20200911T170127_985_TAG_TESTING
Submission status: success
Job ID: 8db24ec0-61ef-49ef-a4d3-9bdef3ae1394
Identifier: SISTER_AVCL_L2B_VEGBIOCHEM_20170507T010011_985_TAG_TESTING
Submission status: success
Job ID: d1a7b4ca-948d-40c1-9e22-3127a3d3fa94
Identifier: SISTER_AVNG_L2B_VEGBIOCHEM_20180429t045252_985_TAG_TESTING
Submission status: success
Job ID: fa913bf9-5185-4490-9205-b0d5f9b94f1a
Identifier: SISTER_DESIS_L2B_VEGBIOCHEM_20190609T195930_985_TAG_TESTING
Submission status: success
Job ID: 860f45b5-cbdf-48ab-aab4-775bf421c25d


## Step 6b. Snow grainsize

In [ ]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2B_GRAINSIZE_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    frcover_id = scene['frcover']['job_id'] 
    frcover_result= [x for x in maap.getJobResult(frcover_id).outputs if x.startswith("s3://s3.") and "FRCOV" in x]
    l2b_frcov =  frcover_result[0]                   
    scene['frcover']['frcover_dataset'] = l2b_frcov

    grainsize_job_response = maap.submitJob(
        algo_id="sister-grainsize",
        version="sister-dev",
        reflectance_dataset=  scene['reflect_correct']['reflectance_dataset'],
        frcov_dataset= scene['frcover']['frcover_dataset'],
        snow_cover = 0.9,
        crid= scene['crid'],
        publish_to_cmr=False,
        cmr_metadata={},
        queue="sister-job_worker-16gb",
        identifier=identifier)
    
    print(f'Identifier: {identifier}')
    print('Submission status: %s' % grainsize_job_response.status)
    print('Job ID: %s' % grainsize_job_response.id)
    scene['grainsize']  = {'job_id' : grainsize_job_response.id}

In [12]:
vegbiochem_job_response = maap.submitJob(
algo_id="sister-trait_estimate",
version="sister-dev",
reflectance_dataset='s3://s3.us-west-2.amazonaws.com:80/sister-ops-workspace/LOM/PRODUCTS/AVNG/L2A_CORFL/2018/05/14/SISTER_AVNG_L2A_CORFL_20180514T042654_987',
frcov_dataset='s3://s3.us-west-2.amazonaws.com:80/sister-ops-workspace/LOM/PRODUCTS/AVNG/L2B_FRCOV/2018/05/14/SISTER_AVNG_L2B_FRCOV_20180514T042654_987',
veg_cover = 0.5,
crid = '987',
publish_to_cmr=False,
cmr_metadata={},
queue="sister-job_worker-16gb",
identifier='SISTER_AVNG_L2B_VEGBIOCHEM_20180514T042654_987')

In [11]:
maap.getJobResult('0eff29c9-2350-40da-be87-e7e58c3f9fd2').outputs


['http://sister-ops-workspace.s3-website.us-west-2.amazonaws.com/LOM/PRODUCTS/AVNG/L2A_CORFL/2018/05/14/SISTER_AVNG_L2A_CORFL_20180514T042654_987',
 's3://s3.us-west-2.amazonaws.com:80/sister-ops-workspace/LOM/PRODUCTS/AVNG/L2A_CORFL/2018/05/14/SISTER_AVNG_L2A_CORFL_20180514T042654_987',
 'https://s3.console.aws.amazon.com/s3/buckets/sister-ops-workspace/LOM/PRODUCTS/AVNG/L2A_CORFL/2018/05/14/SISTER_AVNG_L2A_CORFL_20180514T042654_987/?region=us-east-1&tab=overview',
 'http://sister-ops-workspace.s3-website.us-west-2.amazonaws.com/anonymous/dps_output/sister-reflect_correct/sister-dev/2023/02/08/19/38/36/071295',
 's3://s3.us-west-2.amazonaws.com:80/sister-ops-workspace/anonymous/dps_output/sister-reflect_correct/sister-dev/2023/02/08/19/38/36/071295',
 'https://s3.console.aws.amazon.com/s3/buckets/sister-ops-workspace/anonymous/dps_output/sister-reflect_correct/sister-dev/2023/02/08/19/38/36/071295/?region=us-east-1&tab=overview']